# AI Toolkit by Ostris
## Memecoin Multi-Concept LoRA Training


In [ ]:
!nvidia-smi


In [ ]:
!git clone https://github.com/posaune0423/doom-ai-toolkit.git


Put your image dataset in the `/content/dataset` folder. This repo already includes `content/dataset` with the memecoin data, so no extra setup is required.


In [ ]:
!cd doom-ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt


## Model License
FLUX.1-schnell is Apache 2.0 and does not require a gated Hugging Face token, but you still need to agree to the model terms on Hugging Face and keep a token handy if you plan to access private models. Use the next cell to set `HF_TOKEN` when needed.


In [ ]:
import getpass
import os

hf_token = getpass.getpass('Enter your HF access token (press enter to skip): ')

if hf_token:
    os.environ['HF_TOKEN'] = hf_token
    print('HF_TOKEN environment variable has been set.')
else:
    print('No HF token provided; proceeding without setting HF_TOKEN.')


In [ ]:
import os
import sys
sys.path.append('/content/doom-ai-toolkit')
from toolkit.job import run_job
import yaml
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"


## Setup

This is your config. It is documented pretty well. The configuration is loaded from a YAML file. This will run as is without modification, but feel free to edit as you want.


In [ ]:
%cd doom-ai-toolkit

with open('config/train_memecoin_multi_lora.yaml', 'r') as f:
    job_config = yaml.safe_load(f)
print('Configuration loaded')
print(f"Training: {job_config['config']['name']}")
print(f"Datasets: {len(job_config['config']['process'][0]['datasets'])}")


## Run it

Below does all the magic. Check your folders to the left. Items will be in output/memecoin_multi_lora_v1. In the samples folder, there are periodic samples. This doesn't work great with colab. They will be in /content/doom-ai-toolkit/output


In [ ]:
run_job(job_config)


## Done

Check your output dir and get your LoRA


In [ ]:
from IPython.display import Image, display
import os

samples_dir = 'output/memecoin_multi_lora_v1/samples'
if os.path.exists(samples_dir):
    images = sorted([f for f in os.listdir(samples_dir) if f.endswith('.png')])
    for img in images[-6:]:
        display(Image(filename=os.path.join(samples_dir, img)))
